**Loading of data**

Initial code taken from: https://github.com/pierinim/tutorials/blob/master/GGI_Jan2021/Lecture1/Notebook1_ExploreDataset.ipynb

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [3]:
A = ! ls tutorials/Data/JetDataset
fileIN =  f'tutorials/Data/JetDataset/{A[2]}'
print(fileIN)
print(A)

tutorials/Data/JetDataset/jetImage_7_100p_30000_40000.h5
['jetImage_7_100p_0_10000.h5', 'jetImage_7_100p_10000_20000.h5', 'jetImage_7_100p_30000_40000.h5', 'jetImage_7_100p_40000_50000.h5', 'jetImage_7_100p_50000_60000.h5', 'jetImage_7_100p_60000_70000.h5', 'jetImage_7_100p_70000_80000.h5', 'jetImage_7_100p_80000_90000.h5']


In [4]:
file = h5py.File(fileIN)

In [5]:
print(list(file.keys()))

['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']


In [6]:
jetconstituents = file.get('jetConstituentList')

print(jetconstituents.shape)
NUM_PARTICLES = 1000
NUM_JETS = 10000

# Shape: num_jets x num_particles x 4
fourvectors = jetconstituents[:, :NUM_PARTICLES, :4] # Particles (px, py, pz, E)
fourvectors.shape


(10000, 100, 16)


(10000, 100, 4)

In [6]:
features = file.get('jets')
print(features.shape)
# for feat in features:
#     print(feat)


(10000, 59)


In [7]:
features = file.get('particleFeatureNames')
for feat in features:
    print(feat)



b'j1_px'
b'j1_py'
b'j1_pz'
b'j1_e'
b'j1_erel'
b'j1_pt'
b'j1_ptrel'
b'j1_eta'
b'j1_etarel'
b'j1_etarot'
b'j1_phi'
b'j1_phirel'
b'j1_phirot'
b'j1_deltaR'
b'j1_costheta'
b'j1_costhetarel'
b'j1_pdgid'


In [8]:
# Creating full matrix of inner products
M = np.diag([-1, -1, -1, 1])
masses = np.einsum("npi, ij, npj->np", fourvectors, M, fourvectors)

In [9]:

inner_prods = np.einsum("npi, ij, nqj->npq", fourvectors, M, fourvectors)
print(inner_prods.shape)

(10000, 100, 100)


In [10]:
jet_data = np.array(file.get('jets'))
target = jet_data[:, -6:-1]
data = np.expand_dims(inner_prods, axis=-1)
print(f'Target shape: {target.shape} Data shape: {data.shape}')


Target shape: (10000, 5) Data shape: (10000, 100, 100, 1)


In [11]:
print(np.einsum("ii->i", inner_prods[0]))

[-1.99474511e-04  1.51382091e-02  1.97987621e-02  1.97071930e-02
  1.94787945e-02 -7.07619447e-05  1.96450756e-02  1.94215894e-02
  1.97682416e-02  2.48130463e-01  1.95317536e-02  1.97228822e-02
  2.49051544e-01  2.46331456e-01  2.42878742e-01  2.41106736e-01
  2.48052045e-01  1.89166967e-02  1.95089370e-02  1.99180745e-02
  1.95871783e-02  2.37054535e-01  4.42374403e-06  2.47968874e-01
  5.51335120e-06  1.89164223e-02  1.97135626e-02  4.48942043e-06
  1.88289427e-02 -6.00081376e-06 -5.80419841e-06  1.96257486e-02
  1.96897912e-02  1.97620432e-02  1.94742687e-02  1.94733123e-02
  1.95572942e-02  6.10149634e-07  1.92523559e-02  1.95623538e-02
  1.22577499e-06  1.95718340e-02  2.40283596e-01  1.30061693e-06
  1.77183864e-06 -4.89753351e-07 -9.16488379e-07  1.97372971e-02
  7.40285596e-07  3.45385942e-09  4.83733501e-07 -2.37540178e-07
  2.01516156e-02  1.96992159e-02 -4.57837650e-07  1.97272635e-02
 -2.55336297e-07  1.93448001e-02  1.92207415e-02  1.95818228e-02
 -4.51336222e-07  2.44886

In [12]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data[:NUM_JETS], target[:NUM_JETS], test_size=0.33)

In [13]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



(6700, 100, 100, 1) (3300, 100, 100, 1) (6700, 5) (3300, 5)


**Using the PELICAN architecture**

In [14]:
import tensorflow as tf
from keras.models import Sequential
from layers import Msg, LinEq2v2, LinEq2v0, InputLayer
from keras.layers import Dropout, Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.optimizers import AdamW

from tqdm.keras import TqdmCallback


c:\Users\otkul\ETH\vår\project\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
dropout_rate = 0.02
model = Sequential(layers= [
    InputLayer(),
    Msg(30, activation='relu'),
    Dropout(dropout_rate),
    LinEq2v2(60, activation='relu'),
    Dropout(dropout_rate),
    LinEq2v0(120, activation='relu'),
    Dropout(dropout_rate),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax'),
]
)


model.compile(
    optimizer=AdamW(),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy()],
)



In [ ]:
def scheduler(epoch, lr):
    if epoch < 4:


In [16]:
EPOCHS = 10
BATCH = 128
# model.build((BATCH, 100, 100, 1))



In [17]:

history = model.fit(
    X_train,
    y_train,
    epochs = EPOCHS,
    batch_size = BATCH,
    validation_data=(X_test, y_test),
    callbacks=[TqdmCallback(verbose=1)],
    verbose=0
)


  0%|          | 0/10 [00:00<?, ?epoch/s]

 20%|██        | 2/10 [31:34<2:05:17, 939.74s/epoch, loss=3.26e+5, categorical_accuracy=0.506, val_loss=1.7e+6, val_categorical_accuracy=0.113]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    (None, 100, 100, 1)       0         
                                                                 
 msg (Msg)                   (None, 100, 100, 30)      150       
                                                                 
 dropout (Dropout)           (None, 100, 100, 30)      0         
                                                                 
 lin_eq2v2 (LinEq2v2)        (None, 100, 100, 60)      27000     
                                                                 
 dropout_1 (Dropout)         (None, 100, 100, 60)      0         
                                                                 
 msg_1 (Msg)                 (None, 100, 100, 30)      1920      
                                                                 
 dropout_2 (Dropout)         (None, 100, 100, 30)      0

In [ ]:
print(X_train[0].shape)

index =43

ypred = model(X_test[index:index+1])
y = y_test[index]
print(y)
print(ypred)


(100, 100, 1)
[0. 1. 0. 0. 0.]
tf.Tensor([[1. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
